In [1]:
#!pip install pmdarima
#!pip install TBATS
#!pip install auto_ts
#pip install fedot==0.3.0
# !pip install auto_ts

In [208]:
import os
import pandas as pd; pd.set_option('display.max_columns', None) 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import pickle
from IPython.display import clear_output as co
from random import random
import warnings; # ...

import sklearn.preprocessing
import sklearn.model_selection
import sklearn.metrics
import sklearn.utils

from statsmodels.tsa.ar_model import AutoReg
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.vector_ar.var_model import VAR
from statsmodels.tsa.statespace.varmax import VARMAX
from pmdarima.arima import auto_arima
from tbats import TBATS
# import auto_ts
# import prophet
#from fedot.api.main import Fedot

import tools

In [3]:
le = sklearn.preprocessing.LabelEncoder
#regressors = dict(sklearn.utils.all_estimators('regressor'))
metrics = sklearn.metrics._regression
mae = metrics.mean_absolute_error
mse = metrics.mean_squared_error
mape = metrics.mean_absolute_percentage_error
r2 = metrics.r2_score
me = metrics.max_error
medae = metrics.median_absolute_error
evs = metrics.explained_variance_score
mpd = metrics.mean_poisson_deviance
mgd = metrics.mean_gamma_deviance
def wape(ye, yhat):
    return np.abs(ye-yhat).sum()/ye.sum()
def e(ye, yhat):
    return np.abs(yhat-ye).sum()
def estd(ye, yhat):
    return np.abs(yhat-ye).std()

### Declarando Funções

In [3]:
def load_series(path='series/'):
    filenames = os.listdir(path)
    series, cnt, n_files = {}, 0, len(filenames)
    freq = range(1, n_files, 10)
    for filename in filenames:
        cnt+=1
        if cnt in freq: print(f'Files loaded: {cnt}/{n_files}'); co(wait=True)
        series[filename[:-4]] = pd.read_csv(path+filename, index_col=0)
    print(f'Done! Loaded {n_files} files.')
    return series

## Return random serie's city and product-type
def sample_serie():
    keys = np.random.choice(list(series.keys())).split('-')
    return keys[0], keys[1:]
    
def clean_series(series, train_min=1, drop_start_zeros=True):
    clean_series = {}
    test_empty = []
    excluded_test_index = []
    n_total = len(series)
    n_values_total = 0
    n_values_test = 0
    n_values_train = 0
    n_series = 0
    n_series_empty = 0
    n_series_test_empty = 0
    n_series_train_min_empty = 0
    n_values = 0
    n_test_empty_total = 0
    n_train_empty_total = 0
    n_non_empty_values = 0
    n_train_zeros = 0

    cnt, freq = 0, range(1, n_total, 10)
    for key in list(series.keys()):
        cnt+=1
        if cnt in freq: print(f'Cleaned series: {cnt}/{n_total}'); co(wait=True)

        msk = series[key]['year'].isin(['01/01/2016', '01/01/2017'])
        test = series[key][msk]
        train = series[key][msk==False]

        n = len(series[key])
        n_empty = series[key][target].isna().sum()
        n_test_empty = test[target].isna().sum()
        n_train_empty = train[target].isna().sum()

        cond1 = n_test_empty == len(test)
        cond2 = n_train_empty > (len(train) - train_min) # if number of training samples is greater than specified minimum.

        n_values_total += n
        n_values_test += len(test)
        n_values_train += len(train)

        n_series_empty += n==n_empty
        n_test_empty_total+=n_test_empty
        n_train_empty_total+=n_train_empty
        n_series_test_empty += cond1
        n_series_train_min_empty += cond2

        if cond1 or cond2:
            n_series+=1
            n_values+=len(series[key])
            n_non_empty_values+=(len(series[key]) - n_empty)
            excluded_test_index.extend(test[test[target].isna()==False].index.tolist())

        else:
            n_values+=n_empty
            clean_series[key] = series[key].dropna(subset=['area'])
            if drop_start_zeros:
                test_msk = clean_series[key]['year'].isin(['01/01/2016', '01/01/2017'])
                train_msk = test_msk==False
                isNotZero = (clean_series[key][train_msk]['area']!=0).tolist()
                if sum(isNotZero)!=0:
                    testset = clean_series[key][test_msk]
                    trainset_cut = clean_series[key][train_msk].iloc[isNotZero.index(True):]
                    clean_series[key] = pd.concat([trainset_cut, testset])
                    n_train_zeros+=(sum(train_msk)-len(trainset_cut))


    print('Values Count:')
    print( pd.Series(
        [n_total, n_values_total, n_values_test, n_values_train],
        index=['n_total', 'n_values_total', 'n_values_test', 'n_values_train'])
    )
    print(); print('Excluded count:')
    print(
        pd.Series([
            n_series, n_series_empty, n_series_test_empty, n_series_train_min_empty,
            n_values, n_values-n_non_empty_values, n_test_empty_total, n_train_empty_total,
            n_non_empty_values, len(excluded_test_index), n_train_zeros
        ],
        index=[
            'series', 'series_empty', 'series_test_empty', f'series_train_min_<_{train_min}',
            'values', 'values_empty', 'test_empty', 'train_empty',
            'non_empty_values', 'non_empty_test_values', 'n_train_zeros'
        ])
    )
    return clean_series, excluded_test_index

def test_data():
    Y_e = []
    for key in series.keys():
        x_t, x_e, y_t, y_e = custom_tts(key)
        Y_e.append(y_e)
    return pd.concat(Y_e)

def get_prod_indexes():
    prod_indexes = {}
    for prod in prods:
        if prod not in ['Sorghum', 'Açaí']:
            if '-' in prod: 
                product, prod_type = prod.split('-')
            else:
                product = prod
            df = data_e[data_e['product']==product].copy()        
            if '-' in prod: 
                df = df[df['product_type']==prod_type]

            prod_indexes[prod] = df.index.copy()
    return prod_indexes

def get_prodtype_indexes():
    prodtype_indexes = {}
    for prodtype in ['temporary', 'permanent', 'pasture']:
        df = data_e[data_e['product_type']==prodtype].copy()        
        prodtype_indexes[prodtype] = df.index.copy()
    return prodtype_indexes

#### Mapping between the series test indexes and the series key
def get_key_index_test_map():    
    key_test_index_map = {}
    for key in series.keys():    
        key_index = series[key].index
        for entry_index in key_index:
            if entry_index in Y_e.index:
                key_test_index_map[entry_index] = key
    return key_test_index_map

def get_key_prodtype_map():
    key_prodtype_map = {}
    for key in series.keys():
        key_prodtype_map[key] = series[key].iloc[0]['product_type']
    return key_prodtype_map

#### Extract predictive and target variables
def get_xy(data, target='area', fill_na=0):
    ## 1. Extraindo variáveis preditivas
    X = data.drop(target, 1)
    ## 2. Extraindo Variável Álvo
    Y = data[target].copy()
    #### Substituição de Valores Vazios para Variável Alvo
#     Y[Y.isna()] = fill_na
    return X,Y

#### Label Encode pandas dataframe
def label_encode(X, base=None):
    if type(base)==type(None): base = X
    X_lab = X.copy()
    for column in X:
        X_lab[column] = le().fit(base[column]).transform(X[column])
    return X_lab

## Separação das amostras para treinamento e teste
def split_ts(x, y, col='year', horizon=2):
#     periods = x[col].sort_values().iloc[:len(x)-horizon]
    msk = x[col].isin([42, 43])
    return [x[msk==False], x[msk], y[msk==False], y[msk]]

### Train test split of time series by key
def custom_tts(key, encode=True):
    x, y = get_xy(series[key], target='area', fill_na=0)
    if encode: x = label_encode(x[['year']], base=data)
    return split_ts(x, y, col='year', horizon=2)

## Evaluating Model Predictions
def score_predictions(y_e, y_hat):
    return [mae(y_e, y_hat), mse(y_e, y_hat), mape(y_e, y_hat), wape(y_e, y_hat), r2(y_e, y_hat)]

## Evaluating model predictions given scorer list
def score_predictions_custom(y_e, y_hat, scorers=None):
    if type(scorers)==type(None):
        scorers = [mae, mse, mape, wape, r2]; names=scrs
    return [scorer(y_e, y_hat) for scorer in scorers]

def isComplete(ts):
    return (ts.index.max().year - ts.index.min().year + 1) == len(ts)

def train_test_min_max_year(train, test):
    return train.index.min().year, train.index.max().year, test.index.min().year, test.index.max().year

def train_test_min_max(train, test):
    return train.index.min(), train.index.max(), test.index.min(), test.index.max()


In [24]:
## Evaluating model predictions given scorer list
def score_predictions_fancy(y_e, y_hat, model_name='', scorers=None, names=None):
    if type(scorers)==type(None):
        scorers = [mae, mse, mape, wape, r2]; names=scrs
    scores = [scorer(y_e, y_hat) for scorer in scorers]
    return pd.Series(scores, name=model_name, index=names)

In [145]:
## Evaluating model predictions given scorer list
def score_predictions_custom(y_e, y_hat, scorers=None):
    if type(scorers)==type(None):
        scorers = [mae, mse, mape, wape, r2]; names=['mae', 'mse', 'mape', 'wape', 'r2']
    return [scorer(y_e, y_hat) for scorer in scorers]

def score_by_product(Y_e, Y_hat, indexes, model_name, key='product_type', scorers=None, scorers_names=['mae', 'mse', 'mape', 'wape', 'r2']):
    prod_scrs = []
    for product in indexes.keys():
        prod_index = indexes[product]
        prod_scrs.append([ product ] + score_predictions_custom(Y_e.loc[prod_index], Y_hat.loc[prod_index], scorers=scorers))
    prod_scrs = pd.DataFrame(prod_scrs, columns=[key]+scorers_names)
    return prod_scrs

def score_model_predictions(Y_hat, Y_e, indexes, model_name, by='product_type', scorers=None, scorers_names=['mae', 'mse', 'mape', 'wape', 'r2']):
    model_scrs = pd.Series(score_predictions_custom(Y_e, Y_hat, scorers=scorers), index=scorers_names).to_frame().rename(columns={0: model_name}).T;
    prod_scrs = score_by_product(Y_e, Y_hat, indexes, model_name, key=by, scorers=scorers, scorers_names=scorers_names)
    return model_scrs, prod_scrs

---
### 0.1 Loading data

In [15]:
target = 'area'
target_years = ['01/01/2016', '01/01/2017']
prodtypes = ['permanent', 'temporary', 'pasture']
prods = [
    'Rice', 'Beans', 'Cassava',
    'Corn', 'Soy', 'Sorghum',
    'Cocoa', 'Palm oil', 'Açaí',
    'Livestock', 'Others-temporary', 'Others-permanent'
]

In [6]:
data = pd.read_csv('data/historical-database-clean.csv')
X_full = pd.read_csv('data/raw_series.csv', index_col=0)
X = pd.read_csv('data/series.csv', index_col=0)
raw_series = tools.preprocess.load_series(path='data/series/')
series = tools.preprocess.load_series(path='data/series clean/')
keys = list(series.keys())

Done! Loaded 1107 files.


In [53]:
X_train = X.loc[range(42)].copy()
X_test = X.loc[[42, 43]].copy()

In [151]:
#### Extracting test data (2016 and 2017)
Y_e = tools.preprocess().test_data(series, data, keys=keys)
data_e = data.loc[Y_e.index].copy()
#### Extracting test index for each category in product_type-product combinations
prod_indexes = tools.category_index.get_ctgr_combs_indexes(data_e, prods)
#### Extracting test index for each category in product_type
prodtype_indexes = tools.category_index.get_ctgrs_indexes(data_e)
#### Mapping between the series test indexes and the series key
key_test_index_map = tools.category_index.get_key_index_test_map(series, Y_e)

int_year = {'01/01/2016': 42, '01/01/2017': 43}
key_year_test_index_map = {42: {}, 43: {}}
for index, key in key_test_index_map.items():
    row = data.loc[index]
    key_year_test_index_map[int_year[row['year']]][key] = index

index2016, index2017 = [], []
for key in keys:
    for year, index_list in zip([42, 43], [index2016, index2017]):
        try:
            index_list.append(key_year_test_index_map[year][key])
        except: 
            index_list.append(np.nan)
year_key_index = pd.Series(index2016 + index2017)

x_test = pd.Series(X_test.values.reshape(-1), index=year_key_index).dropna()

### 0.2 Processing data

In [76]:
#### Setting label encoded index
LE = le().fit(data['year'])

lab_series = {}
for key in keys:
    lab_series[key] = series[key].set_index(LE.transform(series[key]['year']))[target].copy()

#### Train test split for label indexed series
tt_lab_series = {}
for key in keys:
    msk = lab_series[key].index.isin([42, 43])
    tt_lab_series[key] = {'train': lab_series[key][msk==False].copy(), 'test': lab_series[key][msk].copy()}

tt_lab_series[keys[0]]['train']

41    55.0
Name: area, dtype: float64

---
## Comparing specialized models

#### Metric criterias settings

In [20]:
criteria_map = {'e': 0, 'estd': 0, 'max_error': 0, 'mae': 0, 'mse': 0, 'medae': 0, 'mape': 0, 'wape': 0, 'r2': -1, 'evs': -1}
scorers_names = ['e','estd','max_error','mae','mse','medae','mape','wape','r2','evs']
scorers = [e, estd, me, mae, mse, medae, mape, wape, r2, evs]
scrs = ['mae', 'mse', 'mape', 'wape', 'r2']
sel_scrs = scorers_names #scrs

#### Sampling serie to test individual models

In [17]:
# contrived train and test datasets
train = tt_lab_series[full_keys[0]]['train']
test = tt_lab_series[full_keys[0]]['test']
# train and test prediction intervals
x_train_min, x_train_max, x_test_min, x_test_max = train_test_min_max(train, test)
# x_train_min_year, x_train_max_year, x_test_min_year, x_test_max_year = train_test_min_max_year(train, test)

In [18]:
x_train_min, x_train_max, x_test_min, x_test_max

(0, 41, 42, 43)

---
## AR example

In [46]:
def AutoReg_predict(train, lags=1, x_min=42, x_max=43):
    if len(train)<=3: return np.array([np.nan for i in range(2)])
    model = AutoReg(train, lags=lags)
    model_fit = model.fit()
    # make prediction
    return model_fit.predict(x_min, x_max)

def AutoReg_predict_series(tt_series, keys, lags=1):
    yhat = {}
    for key in keys:
        co(wait=True); print(f'{keys.index(key)}/{len(keys)}')
        testset = tt_series[key]['test']
        trainset = tt_series[key]['train']
        yhat[key] = AutoReg_predict(trainset.values, lags=lags, x_min=len(trainset), x_max=len(trainset)+1)
        for i, ind in zip([0, 1], [42, 43]):
            if ind not in testset.index:
                yhat[key][i] = np.nan
                
    return pd.DataFrame(yhat)

In [156]:
Yhat_AutoReg = AutoReg_predict_series(tt_lab_series, keys, lags=1)
# yhat = pd.Series(Yhat_AutoReg.values.reshape(-1), index=year_key_index).loc[x_test.index].fillna(0)
# scrs, prod_scrs = score_model_predictions(x_test, yhat, prodtype_indexes, model_name='AR', by='product_type')

In [186]:
ar_scrs, ar_prod_scrs = reshape_score_predictions(
    Yhat_AutoReg, x_test, year_key_index, prodtype_indexes,
    fill_na=0, model_name='AR', col_name='product_type'
)

In [187]:
ar_scrs

,mae,mse,mape,wape,r2
AR,1409.396107,2.949627e+07,8.753407e+17,0.06951,0.996326


In [188]:
ar_prod_scrs

,product_type,mae,mse,mape,wape,r2
0,temporary,936.629170,2.622775e+07,2.495408e+17,0.468343,0.727617
1,permanent,784.828075,1.096750e+07,2.576952e+18,0.712266,0.051423
2,pasture,4860.063332,8.394538e+07,7.417297e-02,0.033993,0.998061


---
## ARIMA example

In [159]:
def arima_predict(train, order=(1,1,1), x_min=42, x_max=43):
    if len(train)<=2: return np.array([np.nan for i in range(2)])
    # fit model
    model = ARIMA(train, order=order)
    model_fit = model.fit()
    # make prediction
    return model_fit.predict(x_min, x_max)

def arima_predict_series(tt_series, keys, order=(1,1,1)):
    yhat_arima = {}
    for key in keys:
        co(wait=True); print(f'{keys.index(key)}/{len(keys)}')
        testset = tt_series[key]['test']
        trainset = tt_series[key]['train']
        yhat_arima[key] = arima_predict(trainset.values, order=order, x_min=len(trainset), x_max=len(trainset)+1)
        for i, ind in zip([0, 1], [42, 43]):
            if ind not in testset.index:
                yhat_arima[key][i] = np.nan
    
    return pd.DataFrame(yhat_arima, index=[42, 43])

In [160]:
Yhat_arima = arima_predict_series(tt_lab_series, keys, order=(1,1,1))

1106/1107


In [197]:
arima_scrs, arima_prod_scrs = reshape_score_predictions(
    Yhat_arima, x_test, year_key_index, prodtype_indexes,
    fill_na=0, model_name='AR', col_name='product_type'
)

In [191]:
arima_scrs

,mae,mse,mape,wape,r2
AR,1181.622571,1.554773e+07,7.301628e+17,0.059485,0.99798


In [192]:
arima_prod_scrs

,product_type,mae,mse,mape,wape,r2
0,temporary,666.260016,7.585131e+06,4.335110e+15,0.391854,0.849011
1,permanent,741.861525,1.082450e+07,2.571330e+18,0.713232,-0.078549
2,pasture,4424.198986,6.118807e+07,6.838385e-02,0.031309,0.998520


## SARIMAX example

In [169]:
# contrived train and test datasets
def predict_score_SARIMAX(series, key, order=(1, 1, 1), seasonal_order=(0, 0, 0, 0), exog_train=None, exog_test=None):
    train = series[key]['train']
    test = series[key]['test']
    if len(train)<=2: return np.array([np.nan for i in range(2)])
    # fit model
    model = SARIMAX(train.values, exog=exog_train, order=order, seasonal_order=seasonal_order)
    model_fit = model.fit(disp=False)
    # make prediction
    yhat = model_fit.predict(len(train), len(train)+1, exog=exog_test)
    for i, ind in zip([0, 1], [42, 43]):
        if ind not in test.index:
            yhat[i] = np.nan
    return yhat

def predict_score_series_SARIMAX(series, keys):
    yhat = []
    for key in keys:
        co(wait=True); print(f'{keys.index(key)}/{len(keys)}')
        yhat_sarimax = predict_score_SARIMAX(tt_lab_series, key, order=(1, 1, 1), seasonal_order=(0,0,0,0))
        yhat.append(yhat_sarimax)
    yhat = pd.DataFrame(yhat).T; yhat.columns=keys
    return yhat

In [170]:
Yhat_sarimax = predict_score_series_SARIMAX(tt_lab_series, keys[:])

1106/1107


In [196]:
sarimax_scrs, sarimax_prod_scrs = reshape_score_predictions(
    Yhat_sarimax, x_test, year_key_index, prodtype_indexes,
    fill_na=0, model_name='AR', col_name='product_type'
)

In [198]:
sarimax_scrs

,mae,mse,mape,wape,r2
AR,1181.622571,1.554773e+07,7.301628e+17,0.059485,0.99798


In [199]:
sarimax_prod_scrs

,product_type,mae,mse,mape,wape,r2
0,temporary,666.260016,7.585131e+06,4.335110e+15,0.391854,0.849011
1,permanent,741.861525,1.082450e+07,2.571330e+18,0.713232,-0.078549
2,pasture,4424.198986,6.118807e+07,6.838385e-02,0.031309,0.998520


## VAR example

In [216]:
def predict_VAR(X_train, steps=2, exog_train=None, exog_test=None):
    # fit model
    model = VAR(X_train, exog=exog_train)
    model_fit = model.fit()
    # make prediction
    yhat = model_fit.forecast(model_fit.y, steps=steps, exog_future=exog_test)
    return pd.DataFrame(yhat, columns=X_train.columns, index=range(len(X_train), len(X_train)+steps))

DatetimeIndex(['1974-01-01', '1975-01-01', '1976-01-01', '1977-01-01',
               '1978-01-01', '1979-01-01', '1980-01-01', '1981-01-01',
               '1982-01-01', '1983-01-01', '1984-01-01', '1985-01-01',
               '1986-01-01', '1987-01-01', '1988-01-01', '1989-01-01',
               '1990-01-01', '1991-01-01', '1992-01-01', '1993-01-01',
               '1994-01-01', '1995-01-01', '1996-01-01', '1997-01-01',
               '1998-01-01', '1999-01-01', '2000-01-01', '2001-01-01',
               '2002-01-01', '2003-01-01', '2004-01-01', '2005-01-01',
               '2006-01-01', '2007-01-01', '2008-01-01', '2009-01-01',
               '2010-01-01', '2011-01-01', '2012-01-01', '2013-01-01',
               '2014-01-01', '2015-01-01', '2016-01-01', '2017-01-01'],
              dtype='datetime64[ns]', freq=None)

In [270]:
# x_train = X.copy(); x_train.index = pd.DatetimeIndex(np.array([str(year) for year in range(1974, 2018)], dtype='datetime64[Y]'))
x_train = X_train.iloc[14:].fillna(0).copy()

In [278]:
model = VAR(x_train)
model_fit = model.fit()
# make prediction
yhat = model_fit.forecast(model_fit.y, steps=2, exog_future=None)

C:\Users\luisr\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


AttributeError: 'VARResults' object has no attribute 'y'

In [ ]:
Yhat_var = predict_VAR(X_train.loc[14:].fillna(0), steps=2)

In [ ]:
var_scrs, var_prod_scrs = reshape_score_predictions(
    Yhat_var, x_test, year_key_index, prodtype_indexes,
    fill_na=0, model_name='AR', col_name='product_type'
)

In [187]:
ar_scrs

,mae,mse,mape,wape,r2
AR,1409.396107,2.949627e+07,8.753407e+17,0.06951,0.996326


In [188]:
ar_prod_scrs

,product_type,mae,mse,mape,wape,r2
0,temporary,936.629170,2.622775e+07,2.495408e+17,0.468343,0.727617
1,permanent,784.828075,1.096750e+07,2.576952e+18,0.712266,0.051423
2,pasture,4860.063332,8.394538e+07,7.417297e-02,0.033993,0.998061


In [230]:
VAR?

Init signature: VAR(endog, exog=None, dates=None, freq=None, missing='none')
Docstring:     
Fit VAR(p) process and do lag order selection

.. math:: y_t = A_1 y_{t-1} + \ldots + A_p y_{t-p} + u_t

Parameters
----------
endog : array_like
    2-d endogenous response variable. The independent variable.
exog : array_like
    2-d exogenous variable.
dates : array_like
    must match number of rows of endog

References
----------
Lütkepohl (2005) New Introduction to Multiple Time Series Analysis
File:           c:\users\luisr\anaconda3\lib\site-packages\statsmodels\tsa\vector_ar\var_model.py
Type:           type
Subclasses:     


## VARMA example (not working for all series  at once)

In [36]:
# fit model
def predict_VARMAX(X_train, steps=2, order=(1, 1)):
    model = VARMAX(X_train, order=order)
    model_fit = model.fit(disp=False)
    # make prediction
    return model_fit.forecast(steps=steps)

In [39]:
Yhat_varma = predict_VARMAX(X_train.iloc[:, :2], steps=2)

C:\Users\luisr\anaconda3\lib\site-packages\statsmodels\tsa\statespace\varmax.py:161: EstimationWarning: Estimation of VARMA(p,q) models is not generically robust, due especially to identification issues.
  warn('Estimation of VARMA(p,q) models is not generically robust,'
C:\Users\luisr\anaconda3\lib\site-packages\statsmodels\base\model.py:567: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warn("Maximum Likelihood optimization failed to converge. "
C:\Users\luisr\anaconda3\lib\site-packages\statsmodels\tsa\statespace\varmax.py:161: EstimationWarning: Estimation of VARMA(p,q) models is not generically robust, due especially to identification issues.
  warn('Estimation of VARMA(p,q) models is not generically robust,'


In [42]:
yhat = Yhat_varma.values.reshape(-1)
x_test = X_test.iloc[:, :2].values.reshape(-1)

In [43]:
score_predictions(x_test, yhat)

[13.14127815762053,
 311.53917647104606,
 0.26099213000069693,
 0.30739831947650365,
 0.6276516901817631]

## VARMAX example

In [ ]:
# fit model
model = VARMAX(X_train.iloc[:, 1:5], exog=X_train.iloc[:, 6:10], order=(1, 1))
model_fit = model.fit(disp=False)

In [1036]:
# make prediction
Yhat_varmax = model_fit.forecast(exog=X_test.iloc[:, 6:10], steps=2)
Yhat_varmax

C:\Users\luisr\anaconda3\lib\site-packages\statsmodels\tsa\statespace\varmax.py:161: EstimationWarning: Estimation of VARMA(p,q) models is not generically robust, due especially to identification issues.
  warn('Estimation of VARMA(p,q) models is not generically robust,'
C:\Users\luisr\anaconda3\lib\site-packages\statsmodels\tsa\statespace\varmax.py:161: EstimationWarning: Estimation of VARMA(p,q) models is not generically robust, due especially to identification issues.
  warn('Estimation of VARMA(p,q) models is not generically robust,'


,0307883b5d063703-Others-temporary,03e477d4ede00e89-Others-permanent,03e477d4ede00e89-Others-temporary,051e9cc7d7636816-Others-permanent
42,6.485532,506.906915,37.889265,15.258532
43,3.001125,504.499522,-11.166915,14.415549


In [853]:
score_predictions_custom(Yhat_varmax.values.reshape(-1), X_test.iloc[:, 1:5].values.reshape(-1))

[45.956374751841096,
 7202.773487870832,
 0.9593998911077435,
 0.3412740858180361,
 0.8436123022543823]

## AutoArima Example

In [ ]:
def predict_series_auto_arima(series, keys, n_periods=2):
    Yhat, scores = [], []
    for key in keys:
        co(wait=True); print(f'{keys.index(key)}/{len(keys)}')
        train = series[key]['train']
        test = series[key]['test']
        if len(train)<=2:
            Yhat.append(pd.Series([np.nan for i in range(2)], index=[42,43], name=key))
            continue
        model = auto_arima(y=train.values, seasonal=False, stepwise=True)
        yhat = model.predict(n_periods=n_periods)
        for i, ind in zip([0, 1], [42, 43]):
            if ind not in test.index:
                yhat[i] = np.nan
        Yhat.append(pd.Series(yhat, index=[42,43], name=key))
#         scores.append(score_predictions_fancy(test, yhat, model_name=key))
    return pd.concat(Yhat, 1)#, pd.concat(scores, 1)

In [177]:
Yhat_autoarima =  predict_series_auto_arima(tt_lab_series, keys)

1106/1107


In [ ]:
score_predictions_fancy(Yhat_autoarima.values.reshape(-1), X_test.values.reshape(-1))

### TBATS Example

In [56]:
def predict_tbats(train, steps):
    model_tbats = TBATS(seasonal_periods=(12, 28),\
                  use_arma_errors=False,\
                  use_box_cox=False,\
                  n_jobs=1,\
                  use_trend=None,\
                  use_damped_trend=None)\
                  .fit(train)

    return model_tbats.forecast(steps=steps)

In [57]:
def predict_series_tbats(series, keys, steps=2):
    Yhat, scores = [], []
    for key in keys:
        co(wait=True); print(f'{keys.index(key)}/{len(keys)}')
        train = series[key]['train']
        test = series[key]['test']
        yhat = predict_tbats(train, steps=len(test))
        Yhat.append(pd.Series(yhat, index=range(len(train), len(train)+steps), name=key))
        scores.append(score_predictions_fancy(test, yhat, model_name=key))
    return pd.concat(Yhat, 1), pd.concat(scores, 1)

In [62]:
Yhat_tbats, scores = predict_series_tbats(tt_lab_series, full_keys[:], steps=2)

285/286


In [63]:
score_predictions_fancy(X_test.iloc[:, :].values.reshape(-1), Yhat_tbats.values.reshape(-1))

mae     4.257729e+02
mse     1.108583e+06
mape    1.397161e+17
wape    8.801666e-01
r2      4.817428e-01
Name: , dtype: float64

---
## Comparing models

In [179]:
model_names = ['Yhat_AutoReg', 'Yhat_arima', 'Yhat_sarimax', 'Yhat_var', 'Yhat_autoarima']
model_predictions = [Yhat_AutoReg, Yhat_arima, Yhat_sarimax, Yhat_var, Yhat_autoarima]

In [187]:
y_hat_i = pd.concat([pd.Series(model.values.reshape(-1), name=model_names[i]) for i, model in enumerate(model_predictions)], 1)
y_hat_i.index = keys*2
y_hat_i['year'] = ['01/01/2016']*len(keys) + ['01/01/2017']*len(keys)
y_hat_i = y_hat_i.reset_index(drop=False).rename(columns={'index': 'key'})

In [192]:
#y_hat_i.to_csv('evaluation/specialized_models_predictions.csv', index=True)

In [70]:
x_test = X_test.values.reshape(-1)
models_scrs = pd.concat([score_predictions_fancy(x_test, y_hat_i[model], model_name=model) for model in y_hat_i], 1)

In [72]:
models_scrs

,Yhat_AutoReg,Yhat_arima,Yhat_sarimax,Yhat_var,Yhat_autoarima,Yhat_tbats
mae,1.150449e+02,1.042907e+02,1.042907e+02,1.293528e+13,1.098114e+02,4.257729e+02
mse,1.168531e+05,1.108632e+05,1.108632e+05,1.031415e+27,1.157460e+05,1.108583e+06
mape,1.643048e+16,9.369222e+15,9.369222e+15,5.444631e+27,9.376776e+15,1.397161e+17
wape,2.378232e-01,2.155918e-01,2.155918e-01,2.674009e+10,2.270043e-01,8.801666e-01
r2,9.453717e-01,9.481720e-01,9.481720e-01,-4.821817e+20,9.458893e-01,4.817428e-01


In [927]:
def plot_preds(Yhat, keys):
    fig = plt.figure(figsize=(12, 10), tight_layout=True)
    n_rows = len(keys)//3
    axes = fig.subplots(n_rows,3)
    cnt = -1
    for axs in axes:
        for i in range(len(axs)):
            if cnt<len(keys)-1:
                ax = axs[i]
                cnt+=1; key=keys[cnt]
                train = tt_lab_series[key]['train']
                test = tt_lab_series[key]['test']
                yhat = Yhat[key]
                train.plot(ax=ax)
                test.plot(ax=ax)
                yhat.plot(ax=ax)
    plt.show()